In [2]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern, hog
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

feature extraction

In [3]:
def get_features(image, radii=None, n_points_multiplier=8, method='uniform', normalize=True):
    if radii is None:
        radii = [1, 2, 3]

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray_image = clahe.apply(gray_image)

    lbp_features = []

    for radius in radii:
        n_points = n_points_multiplier * radius
        lbp_image = local_binary_pattern(gray_image, n_points, radius, method)
        lbp_histogram, _ = np.histogram(
            lbp_image.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2)
        )
        lbp_histogram = lbp_histogram.astype("float") / (lbp_histogram.sum() + 1e-6)
        lbp_features.append(lbp_histogram)

    lbp_feature = np.concatenate(lbp_features)

    hog_features = hog(gray_image, orientations=9, pixels_per_cell=(8, 8), 
                        cells_per_block=(2, 2), block_norm='L2-Hys')


    return np.concatenate((lbp_feature, hog_features))


data processing

In [4]:

def processed_data(data_dir, label_map):
    features, labels = [], []

    for label_name in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label_name)

        if not os.path.isdir(label_path):
            continue

        label_idx = label_map[label_name]

        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            image = cv2.imread(image_path)

            if image is not None:
                features.append(get_features(image))
                labels.append(label_idx)

    return np.array(features), np.array(labels)

label_map = {'Angry': 0, 'Happy': 1, 'Sad': 2}
train_data_dir = r'C:\Users\mahes\OneDrive\Desktop\projects\training'
test_data_dir = r'C:\Users\mahes\OneDrive\Desktop\projects\testing'

X_train, y_train = processed_data(train_data_dir, label_map)
X_test, y_test = processed_data(test_data_dir, label_map)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

pca = PCA(n_components=100) 
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)



model trainig and evalute

In [5]:
def train_and_evaluate_model(model, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"{model_name} Accuracy: {accuracy * 100:.2f}%")
    print(classification_report(y_test, y_pred))

  


classifiers

In [6]:

rf_model = RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_split=5, random_state=42)
train_and_evaluate_model(rf_model, "Random Forest")


svm_params = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'], 'kernel': ['rbf', 'poly']}
svm_model = GridSearchCV(SVC(), svm_params, cv=StratifiedKFold(n_splits=5), scoring='accuracy', n_jobs=-1)
svm_model.fit(X_train, y_train)

best_svm = svm_model.best_estimator_
print("SVM Hyperparameters:", svm_model.best_params_)
train_and_evaluate_model(best_svm, "SVM")


Random Forest Accuracy: 62.69%
              precision    recall  f1-score   support

           0       0.76      0.26      0.39       956
           1       0.62      0.91      0.74      1774
           2       0.59      0.50      0.54      1247

    accuracy                           0.63      3977
   macro avg       0.66      0.56      0.56      3977
weighted avg       0.65      0.63      0.60      3977

SVM Hyperparameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
SVM Accuracy: 69.98%
              precision    recall  f1-score   support

           0       0.58      0.54      0.56       956
           1       0.80      0.83      0.82      1774
           2       0.63      0.63      0.63      1247

    accuracy                           0.70      3977
   macro avg       0.67      0.67      0.67      3977
weighted avg       0.70      0.70      0.70      3977



In [7]:
def process_and_visualize(image_path, save_dir):
    
    os.makedirs(save_dir, exist_ok=True)


    image = cv2.imread(image_path)
    if image is None:
        print("Error: Unable to load image.")
        return


    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

 
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_image = clahe.apply(gray_image)


    clahe_path = os.path.join(save_dir, "clahe_image.jpg")
    cv2.imwrite(clahe_path, clahe_image)


    radius = 2
    n_points = 8 * radius
    lbp_image = local_binary_pattern(clahe_image, n_points, radius, method='uniform')


    lbp_image_normalized = np.uint8(255 * lbp_image / lbp_image.max())
    
    lbp_path = os.path.join(save_dir, "lbp_image.jpg")
    cv2.imwrite(lbp_path, lbp_image_normalized)


    hog_features, hog_image = hog(clahe_image, orientations=9, pixels_per_cell=(8, 8),
                                  cells_per_block=(2, 2), block_norm='L2-Hys', visualize=True)

 
    hog_path = os.path.join(save_dir, "hog_image.jpg")
    hog_image_normalized = np.uint8(255 * hog_image / hog_image.max())
    cv2.imwrite(hog_path, hog_image_normalized)

 
    cv2.imshow("Original Image", gray_image)
    cv2.imshow("CLAHE Enhanced Image", clahe_image)
    cv2.imshow("LBP Image", lbp_image_normalized)
    cv2.imshow("HOG Image", hog_image_normalized)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

  
    print(f"Processed images saved to {save_dir}")


image_path = r"C:\Users\mahes\OneDrive\Desktop\projects\sample.jpg"  
save_dir = r"C:\Users\mahes\OneDrive\Desktop\projects\output.jpg"   
process_and_visualize(image_path, save_dir)


Processed images saved to C:\Users\mahes\OneDrive\Desktop\projects\output.jpg
